**Author:** Beatrice Occhiena s314971. See [`LICENSE`](https://github.com/beatrice-occhiena/Computational_intelligence/blob/main/LICENSE) for details.
- institutional email: `S314971@studenti.polito.it`
- personal email: `beatrice.occhiena@live.it`
- github repository: [https://github.com/beatrice-occhiena/Computational_intelligence.git](https://github.com/beatrice-occhiena/Computational_intelligence.git)

**Resources:** These notes are the result of additional research and analysis of the lecture material presented by Professor Giovanni Squillero for the Computational Intelligence course during the academic year 2023-2024 @ Politecnico di Torino. They are intended to be my attempt to make a personal contribution and to rework the topics covered in the following resources.
- [https://github.com/squillero/computational-intelligence](https://github.com/squillero/computational-intelligence)
- Stuart Russel, Peter Norvig, *Artificial Intelligence: A Modern Approach* [3th edition]
- Richard S. Sutton, Andrew G. Barto, *Reinforcement Learning: An Introduction* [2nd Edition]

.

.

# Lab 10: Reinforcement Learning
**MAIN TASK**: Use reinforcement learning to devise a tic-tac-toe player.
- `Deadline`: Christmas Day 🎅🏻
- `Reviews`: Epiphany 📆

## [1] Tic-Tac-Toe Game Environment

In [3]:
from itertools import combinations
from collections import namedtuple, defaultdict
from random import choice, random
from copy import deepcopy

from tqdm.auto import tqdm
import numpy as np
import matplotlib.pyplot as plt

### Game Class

In [4]:
class TicTacToe:
  def __init__(self, starting_player='X'):
    # Initialize the empty board
    self.board = [[' ' for _ in range(3)] for _ in range(3)]
    # Define magic square to check for a winner
    self.magic_square = [[2, 7, 6], [9, 5, 1], [4, 3, 8]]
    # Define player symbols
    self.players = ['X', 'O']
    # Initialize the current player
    self.current_player = starting_player

  def print_board(self):
    """Display the current state of the board"""
    emoji_map = {'X': '❌', 'O': '🔘', ' ': '⬜'}
    for row in self.board:
      print(''.join([emoji_map[space] for space in row]))
    print()

  def make_move(self, row, col):
    """Make a move on the board if the space is empty"""
    if self.board[row][col] == ' ' and 0 <= row <= 2 and 0 <= col <= 2:
      self.board[row][col] = self.current_player
      return True # Move was successful
    else:
      print("Invalid move!")
      return False # Move was unsuccessful
    
  def switch_player(self):
    """Switch the current player"""
    self.current_player = self.players[(self.players.index(self.current_player) + 1) % 2]

  def check_winner(self):
    """
    Check if there is a winner on the board.
    - Returns 'X' if X wins
    - Returns 'O' if O wins
    - Returns None if there is no winner
    """
    # 1. Convert board positions to magic square values
    x_positions = [self.magic_square[row][col] for row in range(3) for col in range(3) if self.board[row][col] == 'X']
    o_positions = [self.magic_square[row][col] for row in range(3) for col in range(3) if self.board[row][col] == 'O']
    
    # 2. Check if any combination of 3 positions sums to 15
    if any(sum(comb) == 15 for comb in combinations(x_positions, 3)):
      return 'X'
    elif any(sum(comb) == 15 for comb in combinations(o_positions, 3)):
      return 'O'

    return None
  
  def is_draw(self):
    """Check if the game is a draw"""
    return all([space != ' ' for row in self.board for space in row])

### Player Class

In [5]:
class Player:
  def __init__(self):
    pass

  def make_move(self, game: TicTacToe):
    """Make a move in the given TicTacToe game instance"""
    raise NotImplementedError("This method should be implemented by subclasses.")

In [6]:
class HumanPlayer(Player):
  
  def make_move(self, game: TicTacToe):
    """Get a move from the user"""

    # 1. Print the board
    print("Current board:")
    game.print_board()

    # 2. Get a valid move from the user
    valid_move = False
    while not valid_move:
      try:
        row = int(input("Enter row (0-2): "))
        col = int(input("Enter column (0-2): "))
        valid_move = game.make_move(row, col)
        if not valid_move:
          print("Invalid move, try again.")
      except ValueError:
        print("Invalid input. Please enter numbers only.")

In [11]:
class RandomPlayer(Player):
    
  def make_move(self, game: TicTacToe):
    """Make a random move on the board"""
    # 1. Get all empty spaces
    empty_spaces = [(row, col) for row in range(3) for col in range(3) if game.board[row][col] == ' ']
    # 2. Choose a random empty space
    row, col = choice(empty_spaces)
    # 3. Make the move
    game.make_move(row, col)

### Match Implementation

In [7]:
def play_game(game: TicTacToe, player1: Player, player2: Player, verbose=False) -> str:
  
  """
    Play a game of TicTacToe between two players
    - player1 as 'X': The first player to make a move
    - player2 as 'O': The second player to make a move
    
    Returns the winner of the game
  """
  
  # 1. Initialize the game
  game_over = False
  winner = None

  # 2. Play the game
  while not game_over:
    # 2.1. Get the current player
    current_player = player1 if game.current_player == 'X' else player2

    # 2.2. VERBOSE: Print the board
    if verbose:
      print(f"Current player: {game.current_player}")
      game.print_board()

    # 2.3. Make a move
    current_player.make_move(game)

    # 2.4. Check if the game is over
    winner = game.check_winner()
    if winner is not None:
      game_over = True
    elif game.is_draw():
      game_over = True

    # 2.5. Switch the player
    game.switch_player()

  # 3. VERBOSE: Print the final board
  if verbose:
    print("Final board:")
    game.print_board()

  # 4. VERBOSE: Print the winner
  if winner is None:
    if verbose:
      print("It's a draw!")
    return 'draw'
  else:
    if verbose:
      print(f"{winner} wins!")
    return winner

In [8]:
def collect_results(player1: Player, player2: Player, n_games=1000, verbose=False):
  """
    Play n_games of TicTacToe between two players and return the results
    - player1 as 'X': The first player to make a move
    - player2 as 'O': The second player to make a move
  """
  # 1. Initialize the results
  results = defaultdict(int)

  # 2. Play the games
  # tqdm is a progress bar that displays the progress of the loop
  for _ in tqdm(range(n_games)):
    new_game = TicTacToe()
    winner = play_game(new_game, deepcopy(player1), deepcopy(player2), verbose=verbose)
    results[winner] += 1

  # 3. Print the results
  print(f"Player 1 ({player1.__class__.__name__}) wins: {results['X']}")
  print(f"Player 2 ({player2.__class__.__name__}) wins: {results['O']}")
  print(f"Draws: {results['draw']}")

  return results

#### Example matches
Here are some examples of matches between two random players to check the correct implementation of the game mechanics and the results collection.

In [84]:
# 1. Initialize the players
rand_player1 = RandomPlayer()
rand_player2 = RandomPlayer()

# 2. Play one game with verbose output
new_game = TicTacToe()
play_game(new_game, rand_player1, rand_player2, verbose=True)

# 3. Play multiple games to collect the statistics
results = collect_results(rand_player1, rand_player2, n_games=1000, verbose=False)

# 4. Plot the results
plt.title("Random vs Random Stats on 1000 games")
plt.bar(range(3), [results['X'], results['O'], results['draw']], color=['red', 'black', 'gray'])
plt.xticks(range(3), ['X', 'O', 'draw'])
plt.figure(figsize=(5, 5))
plt.show()

Current player: X
⬜⬜⬜
⬜⬜⬜
⬜⬜⬜

Current player: O
⬜⬜⬜
⬜⬜⬜
⬜⬜❌

Current player: X
⬜⬜⬜
⬜⬜🔘
⬜⬜❌

Current player: O
⬜⬜⬜
⬜⬜🔘
⬜❌❌

Current player: X
⬜⬜🔘
⬜⬜🔘
⬜❌❌

Final board:
⬜⬜🔘
⬜⬜🔘
❌❌❌

X wins!


100%|██████████| 1000/1000 [00:00<00:00, 9243.44it/s]

Player 1 (RandomPlayer) wins: 585
Player 2 (RandomPlayer) wins: 308
Draws: 107


<Figure size 500x500 with 0 Axes>

## [2] Reinforcement Learning

### Monte Carlo Method 🤹🏻

The fundamental idea underlying the Monte Carlo method is to use simulations to solve problems that cannot be easily solved analytically.
- **Model-Free**: Unlike other techniques, here we do not assume complete knowledge of the environment's dynamics. Monte Carlo methods require only `experience`, since it addresses uncertainty by `simulating` different possible sequences of events, often referred to as `trajectories`. This makes them particularly useful in situations where model information is difficult to obtain or unavailable.
- **Average Return**: For each sampled trajectory, the method observes the returns obtained by the agent. Returns represent the `cumulative rewards` obtained over the entire sequence of actions. Since different trajectories may result in different returns due to the `stochastic nature` of the environment, the method averages these returns over multiple samples.

  > Action-value function: $Q(s,a) = \frac{1}{N} \sum_{i=1}^{N} G_i(s,a)$
  
  Where $G_i(s,a)$ is the return obtained by the agent after performing action $a$ in state $s$ in the $i$-th sampled trajectory.

The Monte Carlo method is particularly well-suited in this context due to the following reasons:
1. **Limited State Space**: Tic-tac-toe has a finite and relatively small state space, making it feasible for the Monte Carlo method to explore and evaluate all possible states through random sampling. In fact, the more simulations we run, the more accurate our estimates will be. But of course, this comes at the cost of `computational complexity`.
2. **Episodic Nature**: Tic-tac-toe is an episodic task, meaning that each episode (the match) has a well-defined terminal state (win, lose, or draw). This allows us to use the `terminal reward` as a proxy for the value of all states visited in that episode. This approach contrasts with methods that update values or policies at each step within an episode and may result in a form of `delayed learning`, slowing down the adaptation process. However, this is not a problem in our case, since the game has very short trajectories.


#### Parameters
- **Exploration Rate**: $\epsilon$ represents the probability of choosing to explore the environment rather than exploit the current known information. In other words, it's the likelihood that the learning agent will make a random move instead of the move that seems best according to its current knowledge.
- **Discount Factor**: $\gamma$  is a number between 0 and 1 that represents the importance of future rewards compared to immediate rewards. A higher value places more emphasis on future rewards, while a lower value makes the agent more short-sighted, caring more about immediate rewards. In the Monte Carlo method, after each game, we want to update the value of each state encountered during the match. The reward that a state receives depends not only on the immediate reward but also on the rewards of all future states, discounted by this factor. 
  > State value update: $V(s) = V(s) + \alpha [G_t - V(s)]$
  
  > Return from state $s$ onward: $G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + ... = \sum_{k=0}^{\infty} \gamma^k R_{t+k+1}$
- **Learning Rate**: $\alpha$ is the learning rate, which determines the extent to which new information overrides old information. A higher value makes the agent more sensitive to new information, while a lower value makes it more conservative, relying more on past experience.
  

In [17]:
class MonteCarloPlayer(Player):
    def __init__(self, exploration_rate=0.1, discount_factor=0.9, learning_rate=1, exploration_decay=0.95):
        super().__init__()
        self.exploration_rate = exploration_rate
        self.discount_factor = discount_factor
        self.learning_rate = learning_rate
        self.exploration_decay = exploration_decay
        self.state_action_values = defaultdict(lambda: defaultdict(float))  # Nested defaultdict for state-action pairs
        self.state_visits = defaultdict(lambda: defaultdict(int))  # Track visits for each state-action pair
        self.rewards = []  # Track rewards
        self.states = []  # Track states
        self.actions = []  # Track actions

    def choose_action(self, game):
        """
        Choose an action based on the exploration-exploitation tradeoff.
        """
        # Check available actions
        available_actions = [(row, col) for row in range(3) for col in range(3) if game.board[row][col] == ' ']

        # Exploration vs Exploitation
        if random() < self.exploration_rate:
            # Exploration: Choose a random action
            return choice(available_actions)
        else:
            # Exploitation: Choose the best action based on current knowledge
            best_action = None
            best_value = float('-inf')
            for action in available_actions:
                action_value = self.state_action_values[self.serialize_board(game.board)][action]
                if action_value > best_value:
                    best_value = action_value
                    best_action = action
            return best_action

    def make_move(self, game):
        """
        Choose and make a move in the game.
        """
        action = self.choose_action(game)
        row, col = action
        game.make_move(row, col)
        self.states.append(self.serialize_board(game.board))
        self.actions.append(action)

    def serialize_board(self, board):
        """
        Convert the board state to a string representation.
        """
        return ''.join([''.join(row) for row in board])

    def update_rewards(self, result):
        """
        Update rewards based on the game result.
        """
        reward = 1 if result == 'X' else -1 if result == 'O' else 0
        self.rewards = [reward] * len(self.states)

    def update_values(self):
        """
        Update the state-action values based on the outcomes of the game.
        """
        G = 0
        for state, action, reward in zip(reversed(self.states), reversed(self.actions), reversed(self.rewards)):
            G = reward + self.discount_factor * G
            self.state_visits[state][action] += 1
            alpha = 1 / self.state_visits[state][action]
            self.state_action_values[state][action] += alpha * (G - self.state_action_values[state][action])

    def reset(self):
        """
        Reset the history for a new game.
        """
        self.rewards = []
        self.states = []
        self.actions = []

    def end_game(self, result):
        """
        Perform updates at the end of a game.
        """
        self.update_rewards(result)
        self.update_values()
        self.reset()
        self.exploration_rate *= self.exploration_decay

    def train(self, opponent, num_games):
        """
        Train the player by playing a number of games against a given opponent.
        """
        for _ in range(num_games):
            # Initialize a new game
            game = TicTacToe()
            game_over = False

            while not game_over:
                # Determine the current player
                current_player = self if game.current_player == 'X' else opponent

                # Make a move
                current_player.make_move(game)

                # Check for a winner or a draw
                winner = game.check_winner()
                if winner is not None:
                    game_over = True
                    if winner == 'X':
                        self.end_game('win')
                        if opponent.__class__.__name__ == 'MonteCarloPlayer':
                            opponent.end_game('loss')
                    else:
                        self.end_game('loss')
                        if opponent.__class__.__name__ == 'MonteCarloPlayer':
                            opponent.end_game('loss')
                elif game.is_draw():
                    game_over = True
                    self.end_game('draw')
                    if opponent.__class__.__name__ == 'MonteCarloPlayer':
                            opponent.end_game('loss')

                # Switch players
                game.switch_player()

            # Reset the game state for the next game
            self.reset()
            if opponent.__class__.__name__ == 'MonteCarloPlayer':
                opponent.reset()

In [28]:
mc_player = MonteCarloPlayer()
opponent = RandomPlayer()  # or another MonteCarloPlayer

training_steps = [100, 250, 500, 1000, 2500]#, 5000]#, 7500, 10000, 25000, 50000]
training_results_move_first = []
training_results_move_second = []

for steps in tqdm(training_steps):
    
    # 1. Train the RL-agent against the opponent for a given number of steps
    mc_player.train(opponent, steps)

    # 2. Play 1000 games against the opponent making the first move
    results = collect_results(mc_player, opponent, n_games=1000, verbose=False)
    training_results_move_first.append(results)

    # 3. Play 1000 games against the opponent making the second move
    results = collect_results(opponent, mc_player, n_games=1000, verbose=False)
    training_results_move_second.append(results)

# 4. Plot the results
def plot_results(training_results, move_first=True):
    plt.title(f"Monte Carlo Training Results ({'MC' if move_first else 'Random'} Player Moves First)")
    plt.plot(training_steps, [res['X'] for res in training_results], label=('MC' if move_first else 'Random') + ' Player Wins', color='red')
    plt.plot(training_steps, [res['O'] for res in training_results], label=('MC' if move_first else 'Random') + ' Player Losses', color='black')
    plt.plot(training_steps, [res['draw'] for res in training_results], label="Draws", color='gray')
    plt.legend()
    plt.xlabel("Training Steps")
    plt.ylabel("Number of Wins")
    plt.show()

plot_results(training_results_move_first, move_first=True)
plot_results(training_results_move_second, move_first=False)

100%|██████████| 1000/1000 [00:02<00:00, 446.38it/s]


Player 1 (MonteCarloPlayer) wins: 794
Player 2 (RandomPlayer) wins: 171
Draws: 35


 20%|██        | 1/5 [00:04<00:17,  4.34s/it]

Player 1 (RandomPlayer) wins: 519
Player 2 (MonteCarloPlayer) wins: 440
Draws: 41


100%|██████████| 1000/1000 [00:02<00:00, 401.16it/s]


...

Player 1 (RandomPlayer) wins: 533
Player 2 (MonteCarloPlayer) wins: 447
Draws: 20


100%|██████████| 1000/1000 [00:02<00:00, 444.66it/s]


Player 1 (MonteCarloPlayer) wins: 786
Player 2 (RandomPlayer) wins: 170
Draws: 44


100%|██████████| 5/5 [00:23<00:00,  4.71s/it]


Player 1 (RandomPlayer) wins: 498
Player 2 (MonteCarloPlayer) wins: 457
Draws: 45
